In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

import requests

## 공용주차장 면적, 공시지가, 공지기사 년월일 크롤링

In [5]:
df = pd.read_excel("C:/Python_practice/DL_project/gwangjin_contest/dataset/public_parking_lot/공용주차장_recrawling.xlsx", index_col=False)

In [6]:
# 접속 url
url = "https://land.gwangjin.go.kr:449/land/wskras/generalInfo.do#"

# 주소명 검색 홈페이지 접속
path = 'C:/Python_practice/DL_project/chromedriver_win32/chromedriver.exe'
browser = Chrome(executable_path=path) # 개인에게 맞는 경로 수정 필요
browser.get(url)

for index, row in df.iterrows():
    # 법정동 선택
    Select(browser.find_element(By.ID, "selBjdong")).select_by_visible_text(row['행정동'])
    time.sleep(1)

    # 본번 입력
    browser.find_element(By.ID, "bonbeon").send_keys(row['본번'])
    time.sleep(1)

    # 부번 입력
    browser.find_element(By.ID, "bubeon").send_keys(row['부번'])
    time.sleep(1)

    # 검색 버튼 클릭
    browser.find_element(By.ID, "btnSearch").click()
    time.sleep(2)
    
    # 각 요소가 로드될 때까지 대기
    wait = WebDriverWait(browser, 4)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#rdVal2")))
    # wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#kras11Ym")))
    # wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#kras11Jiga")))

    # BeautifulSoup로 페이지 파싱
    soup = BeautifulSoup(browser.page_source, "html.parser")

    # 도로명주소
    address_type_element = soup.select_one("#newAddressInfo option")
    no_address_type_elements = browser.find_elements(By.CSS_SELECTOR, "#newAddressInfo")

    if address_type_element and address_type_element.text:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#newAddressInfo option")))
        df.loc[index, '도로명주소(크롤링)'] = address_type_element.text
    elif no_address_type_elements and no_address_type_elements[0].text == "" :
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#newAddressInfo")))
        df.loc[index, '도로명주소(크롤링)'] = np.nan
    elif address_type_element and address_type_element.text == "도로명주소 정보가 없습니다" :
        df.loc[index, '도로명주소(크롤링)'] = np.nan


    # 토지면적
    df.loc[index, '토지면적'] = soup.select_one("#rdVal2").text

    # 주용도(건물유형)
    building_type_element = soup.select_one("#tblBuildListTab0 tbody tr td div.nowrap")
    no_building_data_elements = browser.find_elements(By.CSS_SELECTOR, "#tblBuildListTab0 tbody tr td[id='bldrgstListTab']")

    if building_type_element and building_type_element.text:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tblBuildListTab0 tbody tr td div.nowrap")))
        df.loc[index, '주용도(건물유형)'] = building_type_element.text
    elif no_building_data_elements and no_building_data_elements[0].text == "조회된 내용이 없습니다.":
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tblBuildListTab0 tbody tr td[id='bldrgstListTab']")))
        df.loc[index, '주용도(건물유형)'] = np.nan


    # 공시지가_기준년월
    date_type_element = soup.select_one("#kras11Ym")
    no_land_price_type_element = browser.find_elements(By.CSS_SELECTOR, "#kras11")

    if date_type_element and date_type_element.text:
        df.loc[index, '공시지가_기준년월'] = date_type_element.text
    elif no_land_price_type_element and no_land_price_type_element[0].text == "공시지가 확인서 요청시 요청한 주소에 대한 지가정보가 없습니다.":
        df.loc[index, '공시지가'] = np.nan
    


    # 공시지가
    land_price_type_element = soup.select_one("#kras11Jiga")
    no_land_price_type_element = browser.find_elements(By.CSS_SELECTOR, "#kras11")

    if land_price_type_element and land_price_type_element.text:
        df.loc[index, '공시지가'] = land_price_type_element.text
    elif no_land_price_type_element and no_land_price_type_element[0].text == "공시지가 확인서 요청시 요청한 주소에 대한 지가정보가 없습니다.":
        df.loc[index, '공시지가'] = np.nan

    # 본번 및 부번 입력란 초기화
    browser.find_element(By.ID, "bonbeon").clear()
    browser.find_element(By.ID, "bubeon").clear()
    time.sleep(1)

# 드라이버 종료
browser.quit()

# # 저장
df.to_excel("C:/Python_practice/DL_project/gwangjin_contest/public_parking_lot_completed_v4.xlsx", index=False)

C:\Users\dohee\AppData\Local\Temp\ipykernel_8052\241069978.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = Chrome(executable_path=path) # 개인에게 맞는 경로 수정 필요


In [26]:
df = pd.read_excel('C:/Python_practice/DL_project/gwangjin_contest/dataset/광진구_지번_추가(786).xlsx')
df

,광진구_지번추가 786개,F,S
0,서울특별시 광진구 광장동 102,서울특별시 광진구 광장동,102
1,서울특별시 광진구 광장동 105,서울특별시 광진구 광장동,105
2,서울특별시 광진구 광장동 110,서울특별시 광진구 광장동,110
3,서울특별시 광진구 광장동 112,서울특별시 광진구 광장동,112
4,서울특별시 광진구 광장동 126,서울특별시 광진구 광장동,126
...,...,...,...
781,서울특별시 광진구 화양동 31-35,서울특별시 광진구 화양동,31-35
782,서울특별시 광진구 화양동 31-50,서울특별시 광진구 화양동,31-50
783,서울특별시 광진구 화양동 32-12,서울특별시 광진구 화양동,32-12
784,서울특별시 광진구 화양동 523-2,서울특별시 광진구 화양동,523-2


In [27]:
# apply를 사용하여 'F+S' 열 생성
df['F+S'] = df.apply(lambda row: row['F'] + "  " + str(row['S']), axis=1)

In [30]:
# 파일 저장 시, 파일 닫고 진행
df.to_excel('C:/Python_practice/DL_project/gwangjin_contest/dataset/광진구_지번_추가(786).xlsx', index=False, sheet_name='new')